In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP
logSH = pd.read_csv(r'F:\data\20200818\logs_20200818_zt_88_03_day_88data\mdLog_SH_20200818_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]
logSH1 = pd.read_csv(r'F:\data\20200818\full_20200818_zt_88_03_day_testprevde\mdLog_SH_20200818_1055.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]

# # 1. data format
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

original data source type: 
[22 23]
new data source type: 
[23 22]
['09:02:46.460' '09:02:46.000' '09:02:46.990' ... '15:23:37.800'
 '15:23:37.000' '15:23:38.000']
['10:54:39.000' '10:54:40.000' '10:54:40.300' ... '15:48:18.070'
 '15:48:19.000' '15:48:19.950']


In [2]:
logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
# logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).ljust(9, "0")))
# logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).ljust(9, "0")))

display(logSH[(logSH["StockID"] == 600004) & (logSH["time"] >= 143000000) & (logSH["time"] <= 145000000)])
display(logSH1[(logSH1["StockID"] == 600004) & (logSH1["time"] >= 143000000) & (logSH1["time"] <= 145000000)])

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
12458904,1597732229768258,204606919,23,600004,SH,143000000,23944566,3.604573e+08,15.10,15.09,...,15.11,15.12,15.13,15.14,13400,43100,4278,2600,17685,15.04
12459097,1597732230213627,204616942,22,600004,SH,143001390,23944566,3.604573e+08,15.10,15.09,...,15.11,15.12,15.13,15.14,13400,43100,4278,2600,17685,15.04
12461790,1597732232752210,204679345,23,600004,SH,143003000,23956466,3.606369e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,14300,43100,4278,2600,17685,15.04
12462106,1597732233213644,204690939,22,600004,SH,143004390,23956466,3.606369e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,14300,43100,4278,2600,17685,15.04
12464528,1597732235622283,204735312,23,600004,SH,143006000,23958466,3.606671e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,37700,19800,3378,2600,17685,15.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13601626,1597733423774875,222373275,23,600004,SH,144954000,26000666,3.914859e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38600,56678,7500,42700,7800,15.04
13602057,1597733424287215,222381978,22,600004,SH,144955450,26000666,3.914859e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38600,56678,7500,12700,7800,15.04
13604402,1597733426655125,222413390,23,600004,SH,144957000,26001066,3.914920e+08,15.07,15.06,...,15.08,15.09,15.10,15.11,38200,56678,7500,12800,7800,15.04
13604928,1597733427287921,222424778,22,600004,SH,144958450,26001166,3.914935e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38200,56678,7500,12800,7800,15.04


,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
6853183,1597732229766529,15484607,23,600004,SH,143000000,23944566,3.604573e+08,15.10,15.09,...,15.11,15.12,15.13,15.14,13400,43100,4278,2600,17685,15.04
6853370,1597732230213727,15486032,22,600004,SH,143001390,23944566,3.604573e+08,15.10,15.09,...,15.11,15.12,15.13,15.14,13400,43100,4278,2600,17685,15.04
6855991,1597732232751366,15496292,23,600004,SH,143003000,23956466,3.606369e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,14300,43100,4278,2600,17685,15.04
6856289,1597732233213951,15497928,22,600004,SH,143004390,23956466,3.606369e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,14300,43100,4278,2600,17685,15.04
6858642,1597732235617296,15505802,23,600004,SH,143006000,23958466,3.606671e+08,15.09,15.09,...,15.11,15.12,15.13,15.14,37700,19800,3378,2600,17685,15.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7961108,1597733423773888,18293870,23,600004,SH,144954000,26000666,3.914859e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38600,56678,7500,42700,7800,15.04
7961523,1597733424287249,18294937,22,600004,SH,144955450,26000666,3.914859e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38600,56678,7500,12700,7800,15.04
7963793,1597733426655974,18300160,23,600004,SH,144957000,26001066,3.914920e+08,15.07,15.06,...,15.08,15.09,15.10,15.11,38200,56678,7500,12800,7800,15.04
7964301,1597733427288101,18301709,22,600004,SH,144958450,26001166,3.914935e+08,15.06,15.06,...,15.08,15.09,15.10,15.11,38200,56678,7500,12800,7800,15.04


In [3]:
in_dex = [16, 300, 852, 905]
data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH["time"] <= 150100000) 
              & (logSH['source'] == 23) & (logSH['time'] >= 105439000)]
data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0) & (logSH1["time"] <= 150100000)
              & (logSH1['source'] == 23)]

columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())

print(n1)
print(n2)
print(set(data1_1['StockID'].unique()) - set(data2_1['StockID'].unique()))
print(set(data2_1['StockID'].unique()) - set(data1_1['StockID'].unique()))
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
print(data1_1['StockID'].nunique())

# data2_1['cum_amount'] = data2_1['cum_amount'].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["time_x"].count()
n2 = test["time_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["time_y"])])
    print(len(test[np.isnan(test["time_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["time_x"])])
    display(n2-n1)
    print((n2-n1)/n1)

1664
1612
{688256, 605188, 688004, 688518, 605318, 688520, 605199, 688528, 603408, 688277, 688027, 688157, 688418, 688165, 605222, 688555, 688558, 688561, 688050, 688180, 688309, 605108, 688311, 688312, 688567, 688568, 601399, 688060, 605118, 688579, 688580, 688069, 600956, 688586, 688589, 688077, 603087, 688336, 688981, 688599, 688600, 605399, 601827, 688377, 605158, 688488, 688106, 605166, 601456, 688500, 688505, 688508}
set()
1612
3733639
3722207
3733639
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount,close,...,ask3q,ask4q,ask5q,openPrice,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y
0,5381098,1597719307782851,107312346,23,600810,SH,105439000,12967022,1.160487e+08,9.01,...,5400,20890,30700,8.75,NaN,NaN,NaN,NaN,NaN,NaN
1,5381100,1597719307805667,107312811,23,688081,SH,105439000,307354,1.471393e+07,48.08,...,1400,2686,3000,47.78,NaN,NaN,NaN,NaN,NaN,NaN
2,5381101,1597719307805683,107312812,23,601618,SH,105439000,40545100,1.188004e+08,2.92,...,2172700,2596400,1185520,2.94,NaN,NaN,NaN,NaN,NaN,NaN
3,5381102,1597719307805694,107312813,23,603486,SH,105439000,1326322,5.205557e+07,38.52,...,200,300,300,40.20,NaN,NaN,NaN,NaN,NaN,NaN
4,5381103,1597719307805706,107312814,23,600392,SH,105439000,24262027,2.197679e+08,8.98,...,60100,22300,17711,9.09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23608,5433624,1597719366824176,108021470,23,603936,SH,105538000,7848255,1.248079e+08,15.91,...,49220,29700,39300,15.83,NaN,NaN,NaN,NaN,NaN,NaN
23723,5433954,1597719367044630,108024722,23,600377,SH,105538000,1835000,1.867514e+07,10.16,...,7100,37400,12100,10.23,NaN,NaN,NaN,NaN,NaN,NaN
23753,5433986,1597719367103513,108025097,23,688139,SH,105538000,3419656,2.660975e+08,76.78,...,5000,6890,118,74.23,NaN,NaN,NaN,NaN,NaN,NaN
23920,5434649,1597719367444003,108028862,23,603786,SH,105538000,462583,3.523174e+07,75.33,...,200,800,1000,75.50,NaN,NaN,NaN,NaN,NaN,NaN


0.0030618921647218703


In [7]:
logSH1[ (logSH1['source'] == 23)].groupby('StockID')['time'].min().min()

105439000

In [8]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

703.0
1436.0
4531.0
1498.0
3676.0
6367.0
0.5637365143851484
0.4361681120904882


In [21]:
logSH[logSH["StockID"].isin(in_dex)& (logSH["time"] <= 150100000)].groupby('StockID')['time'].max()

StockID
16     150056050
300    150056150
852    150056160
905    150056160
Name: time, dtype: int64

In [20]:
in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['time'] >= 105439000)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]
columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

# for cols in ["cum_amount"]:
#     data1_1[cols] = data1_1[cols].round(2)
#     data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])

7342
7338
7342


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount,close,...,ask1p_y,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y
0,5381121,1597719307854634,107313057,22,16,SH,105439160,24731172,4.877568e+10,3360.3267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5381261,1597719307937693,107313893,22,300,SH,105439240,101761386,1.665634e+11,4800.1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5381275,1597719307975154,107314284,22,852,SH,105439250,111276618,1.348357e+11,7372.6667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5381276,1597719307975164,107314285,22,905,SH,105439250,101637491,1.093290e+11,6765.9417,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

494.0
771.0
970.0
651.0
2632.5
6119.5
0.7489779231398201
0.25074952303079856


In [1]:
import pandas as pd
import random
import numpy as np
import pickle 
TradeLog = pd.read_csv(r'F:\data\20200818\logs_20200818_zt_88_03_day_88data\mdTradeLog_20200818_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "TransactTime", "SecurityID", "exchId", 
                                                 "ApplSeqNum", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]]
TradeLog1 = pd.read_csv(r'F:\data\20200818\full_20200818_zt_88_03_day_testprevde\mdTradeLog_20200818_1055.csv').loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
TradeLog1 = TradeLog1[TradeLog1['exchId'] == 1]
TradeLog = TradeLog[TradeLog['exchId'] == 1]
display(set(TradeLog['SecurityID'].unique()) - set(TradeLog1['SecurityID'].unique()))
display(set(TradeLog1['SecurityID'].unique()) - set(TradeLog['SecurityID'].unique()))
stocks = list(set(TradeLog1["SecurityID"].unique()) & set(TradeLog["SecurityID"].unique()))
display(len(stocks))
SH = TradeLog[TradeLog["SecurityID"].isin(stocks)]
SH1 = TradeLog1[TradeLog1["SecurityID"].isin(stocks)]
del TradeLog
del TradeLog1
print(len(SH["SecurityID"].unique()))
print(len(SH1["SecurityID"].unique()))


columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


{600956,
 601399,
 601456,
 601827,
 603087,
 603408,
 605108,
 605118,
 605158,
 605166,
 605188,
 605199,
 605222,
 605318,
 605399,
 688004,
 688027,
 688050,
 688060,
 688069,
 688077,
 688106,
 688157,
 688165,
 688180,
 688256,
 688277,
 688309,
 688311,
 688312,
 688336,
 688377,
 688418,
 688488,
 688500,
 688505,
 688508,
 688518,
 688520,
 688528,
 688555,
 688558,
 688561,
 688567,
 688568,
 688579,
 688580,
 688586,
 688589,
 688599,
 688600,
 688981}

set()

1612

1612
1612
23571699
11394389
23571699
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,TransactTime,SecurityID,exchId_x,ApplSeqNum,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,exchId_y
0,1597713928951067,3540330,92500020,600030,1,1,F,N,330000,100,33000000,27660,55792,NaN,NaN,NaN
1,1597713928951071,3540331,92500020,600030,1,2,F,N,330000,100,33000000,27660,57188,NaN,NaN,NaN
2,1597713928951073,3540332,92500020,600030,1,3,F,N,330000,1000,330000000,27660,92459,NaN,NaN,NaN
3,1597713928951075,3540333,92500020,600030,1,4,F,N,330000,700,231000000,57201,92459,NaN,NaN,NaN
4,1597713928951076,3540334,92500020,600030,1,5,F,N,330000,100,33000000,57202,92459,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12177305,1597719308528227,107321217,105439660,600022,1,2433999,F,S,14100,9992,140887200,2779837,4166144,NaN,NaN,NaN
12177306,1597719308528228,107321218,105439690,601139,1,2434000,F,B,82600,1600,132160000,4166152,4165439,NaN,NaN,NaN
12177307,1597719308528232,107321220,105439720,601519,1,2434002,F,B,115000,1100,126500000,4166156,4163173,NaN,NaN,NaN
12177308,1597719308528233,107321221,105439720,600133,1,2434003,F,S,69800,500,34900000,4165965,4166169,NaN,NaN,NaN


12177310
[ 92500000  92500010  92500020 ... 105439720 105439730 105439740]
1612
[600030 601628 600325 ... 600539 600265 600243]


In [2]:
SH1['TransactTime'].min()

105439740

In [3]:
re = pd.merge(SH[SH['TransactTime'] >= 105439740], SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

11394405
11394389
11394405
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,TransactTime,SecurityID,exchId_x,ApplSeqNum,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,exchId_y
0,1597719308527859,107320994,105439740,600543,1,2338469,F,B,112300,10000,1123000000,3988367,3988280,NaN,NaN,NaN
1,1597719308527860,107320995,105439740,603035,1,2338470,F,S,162200,500,81100000,3812666,3988368,NaN,NaN,NaN
2,1597719308527861,107320996,105439740,600332,1,2338471,F,B,349800,100,34980000,3988370,3769426,NaN,NaN,NaN
3,1597719308527863,107320997,105439740,600183,1,2338472,F,S,281300,900,253170000,3986024,3988371,NaN,NaN,NaN
4,1597719308527865,107320998,105439740,600477,1,2338473,F,S,51400,1000,51400000,3988247,3988372,NaN,NaN,NaN
5,1597719308527866,107320999,105439740,600477,1,2338474,F,S,51300,1000,51300000,3924133,3988372,NaN,NaN,NaN
6,1597719308527868,107321000,105439740,600584,1,2338475,F,B,466000,200,93200000,3988375,3959110,NaN,NaN,NaN
7,1597719308527869,107321001,105439740,603035,1,2338476,F,S,162200,200,32440000,3812666,3988378,NaN,NaN,NaN
8,1597719308527871,107321002,105439740,603035,1,2338477,F,S,162200,300,48660000,3816160,3988378,NaN,NaN,NaN
9,1597719308527872,107321003,105439740,603035,1,2338478,F,S,162200,100,16220000,3816522,3988378,NaN,NaN,NaN


16
[105439740]
8
[600543 603035 600332 600183 600477 600584 600303 600886]


In [4]:
for i in re[np.isnan(re["sequenceNo_y"]) & (re['TransactTime'] == 105439740 )]['SecurityID'].unique():
    try:
        assert(SH1[SH1['SecurityID'] == i]['TransactTime'].min() > 105439740 )
    except:
        print(SH1[(SH1['SecurityID'] == i) & (SH1['TransactTime'] <= 105439740 )].shape[0])
        print(SH[(SH['SecurityID'] == i) & (SH['TransactTime'] <= 105439740 )].shape[0])

In [5]:
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

368.0
594.0
870.0
459.0
1407.0
5554.0
0.6310569175758349
0.36866610399206134
